In [3]:
import pandas as pd
import re
import numpy as np

# Load dữ liệu
df = pd.read_csv('data_full.csv', header=1, encoding='latin1')

def extract_cpu_info(row):
    # Ưu tiên lấy từ cpu_detail, nếu thiếu thì tìm trong title
    text_source = str(row['cpu_detail']) if pd.notna(row['cpu_detail']) and str(row['cpu_detail']).strip() != '' else str(row['title'])
    text = text_source.upper()
    
    # Giá trị mặc định
    info = {
        'cpu_brand': 'Unknown',
        'cpu_family': 'Unknown',
        'cpu_gen': 0,          # Thế hệ mặc định là 0 nếu không tìm thấy
        'cpu_performance': 0.0 # Điểm hiệu năng
    }
    
    # 1. XÁC ĐỊNH HÃNG (BRAND)
    # Dùng cột cpu_brand có sẵn hoặc tìm trong text
    if pd.notna(row['cpu_brand']):
        info['cpu_brand'] = row['cpu_brand']
    elif 'INTEL' in text or 'CORE' in text or 'I5' in text:
        info['cpu_brand'] = 'Intel'
    elif 'AMD' in text or 'RYZEN' in text:
        info['cpu_brand'] = 'AMD'
    elif 'MACBOOK' in text or 'APPLE' in text or 'M1' in text or 'M2' in text:
        info['cpu_brand'] = 'Apple'

    # 2. TRÍCH XUẤT FAMILY & GEN (Dùng Regex đa dạng)
    
    # Pattern cho Intel: Core i5-12400, i7 13620H, Core Ultra 7
    match_intel = re.search(r'(CORE|I)\s*[-]?\s*(I[3579]|ULTRA\s*[579])[- ]?(\d{2,5})?([A-Z]*)', text)
    
    # Pattern cho AMD: Ryzen 5 7535HS, R7-5800H
    match_amd = re.search(r'(RYZEN|R)\s*[-]?\s*([3579])[- ]?(\d{3,4})?([A-Z]*)', text)
    
    # Pattern cho Apple: M1, M2 Pro...
    match_apple = re.search(r'(M[123])\s*(PRO|MAX|ULTRA)?', text)

    suffix = "" # Hậu tố chip (H, U, HX...) để tính điểm hiệu năng
    
    if info['cpu_brand'] == 'Apple' and match_apple:
        info['cpu_family'] = match_apple.group(1) # M1, M2
        info['cpu_gen'] = int(match_apple.group(1)[-1]) # Gen 1, 2
        suffix = match_apple.group(2) if match_apple.group(2) else ""

    elif match_intel:
        # Xử lý Intel
        info['cpu_brand'] = 'Intel' # Cập nhật chắc chắn
        fam_code = match_intel.group(2).replace(" ", "") # I5, ULTRA7
        info['cpu_family'] = f"Core {fam_code}"
        
        # Xử lý Gen
        model_num = match_intel.group(3)
        if model_num:
            if len(model_num) >= 4: # VD: 12400 -> Gen 12, 8250 -> Gen 8
                info['cpu_gen'] = int(model_num[:2]) if len(model_num) == 5 else int(model_num[0])
                if info['cpu_gen'] == 1 and len(model_num) == 4: info['cpu_gen'] = 1 # Gen 1st (hiếm)
                if len(model_num) == 5: info['cpu_gen'] = int(model_num[:2]) # 12xxx -> 12
            else:
                 info['cpu_gen'] = 0
        suffix = match_intel.group(4) if match_intel.group(4) else ""

    elif match_amd:
        # Xử lý AMD
        info['cpu_brand'] = 'AMD'
        info['cpu_family'] = f"Ryzen {match_amd.group(2)}"
        
        # Xử lý Gen (VD: 5600 -> Gen 5, 7xxx -> Gen 7)
        model_num = match_amd.group(3)
        if model_num:
            info['cpu_gen'] = int(model_num[0])
        suffix = match_amd.group(4) if match_amd.group(4) else ""

    # 3. TÍNH ĐIỂM HIỆU NĂNG (SCORING)
    # Bảng điểm cơ bản cho từng Family
    base_score = {
        'Core I3': 30, 'Ryzen 3': 30,
        'Core I5': 50, 'Ryzen 5': 50,
        'Core I7': 70, 'Ryzen 7': 70,
        'Core I9': 90, 'Ryzen 9': 90,
        'Core Ultra 5': 65, 'Core Ultra 7': 85,
        'M1': 60, 'M2': 75, 'M3': 85
    }
    
    score = base_score.get(info['cpu_family'], 40) # Mặc định 40 nếu Unknown
    
    # Cộng điểm thế hệ (Mỗi thế hệ mới +3 điểm)
    score += info['cpu_gen'] * 3
    
    # Nhân hệ số Hậu tố (Suffix Multiplier)
    # HX/Max: Rất mạnh, U: Tiết kiệm điện
    suffix_u = suffix.upper()
    multiplier = 1.0
    if 'HX' in suffix_u or 'MAX' in suffix_u: multiplier = 1.5
    elif 'H' in suffix_u or 'HS' in suffix_u or 'PRO' in suffix_u: multiplier = 1.3
    elif 'P' in suffix_u: multiplier = 1.15
    elif 'U' in suffix_u: multiplier = 0.9
    
    info['cpu_performance'] = round(score * multiplier, 2)
    
    return pd.Series(info)

# Áp dụng vào DataFrame
new_cols = df.apply(extract_cpu_info, axis=1)
df_final = pd.concat([df, new_cols], axis=1)

# Hiển thị kết quả kiểm tra
print(df_final[['title', 'cpu_family', 'cpu_gen', 'cpu_brand', 'cpu_performance']].head())

# Lưu file kết quả
df_final.to_csv('data_encoded_cpu.csv', index=False)

                                               title cpu_family  cpu_gen  \
0          Laptop ASUS TUF Gaming F16 FX607VJ-RL034W    Unknown        0   
1  Laptop Gaming Acer Nitro V 15 ProPanel ANV15-4...    Ryzen 5        7   
2     Laptop ASUS Gaming Vivobook 16X K3605VC-RP431W    Unknown        0   
3                      Laptop HP 14-EP0112TU 8C5L1PA    Core I5        0   
4                Laptop Lenovo V15 G4 IRU 83A100PMVN    Unknown        0   

  cpu_brand cpu_brand  cpu_performance  
0     Intel     Intel             40.0  
1       AMD       AMD             92.3  
2     Intel     Intel             40.0  
3     Intel     Intel             50.0  
4     Intel     Intel             40.0  


In [4]:
df_encoding_cpu_detail = pd.read_csv("data_encoded_cpu.csv", header = 1, encoding = 'latin1')
df_encoding_cpu_detail.head()

,https://cellphones.com.vn/laptop-asus-tuf-gaming-f16-fx607vj-rl034w.html,Laptop ASUS TUF Gaming F16 FX607VJ-RL034W,22.490.000?,24.490.000?,ASUS,Intel,Unnamed: 6,8,12,16,...,NVIDIA GeForce RTX 3050 6GB GDDR6\r\nIntel Iris Xe Graphics,16 inches,1920 x 1200 pixels (WUXGA),"?? sÃ¡ng 300nits, ?? ph? mÃ u NTSC 45%, MÃ n hÃ¬nh ch?ng chÃ³i","56WHrs, 4S1P, 4-cell Li-ion",23,Intel.1,Unknown,0,40.0
0,https://cellphones.com.vn/laptop-acer-gaming-n...,Laptop Gaming Acer Nitro V 15 ProPanel ANV15-4...,25.990.000?,26.990.000?,Gaming,AMD,Ryzen 5 7535HS,6,12,16,...,"NVIDIA GeForce RTX 4050 6 GB GDDR6 VRAM, h? tr...",15.6 inches,1920 x 1080 pixels (FullHD),"?? sÃ¡ng 300nits, ?? ph? mÃ u sRGB 100%, MÃ n ...","57Whr 4-cell Li-ion Battery, 135W AC Adapter",25,AMD,Ryzen 5,7,92.3
1,https://cellphones.com.vn/laptop-asus-vivobook...,Laptop ASUS Gaming Vivobook 16X K3605VC-RP431W,19.490.000?,22.090.000?,ASUS,Intel,NaN,8,12,16,...,NVIDIA Geforce RTX 3050 4GB GDDR6\r\nIntel UHD...,16 inches,1920 x 1200 pixels (WUXGA),"?? sÃ¡ng 300nits, ?? ph? mÃ u 45% NTSC, MÃ n h...","50WHrs, 3S1P, 3-cell Li-ion",28,Intel,Unknown,0,40.0
2,https://cellphones.com.vn/laptop-hp-14-ep0112t...,Laptop HP 14-EP0112TU 8C5L1PA,15.490.000?,19.590.000?,HP,Intel,Core i5 -,10,12,16,...,Intel Iris Xe Graphics,14 inches,1920 x 1080 pixels (FullHD),"MÃ n hÃ¬nh ch?ng chÃ³i, ?? sÃ¡ng 250 nits, ?? ...","3-cell, 41 Wh Li-ion",19,Intel,Core I5,0,50.0
3,https://cellphones.com.vn/laptop-lenovo-v15-g4...,Laptop Lenovo V15 G4 IRU 83A100PMVN,14.990.000?,15.990.000?,Lenovo,Intel,NaN,8,12,16,...,Intel UHD Graphics,15.6 inches,1920 x 1080 pixels (FullHD),"?? sÃ¡ng 300nits, MÃ n hÃ¬nh ch?ng chÃ³i, ?? p...","38Wh, 65W Round Tip (3-pin)",25,Intel,Unknown,0,40.0
4,https://cellphones.com.vn/laptop-lenovo-loq-15...,Laptop Lenovo LOQ 15IAX9E 83LK0079VN,20.790.000?,23.990.000?,Lenovo,Intel,NaN,NaN,NaN,16,...,"NVIDIA GeForce RTX 3050 6GB GDDR6, Boost Clock...",15.6 inches,1920 x 1080 pixels (FullHD),"?? sÃ¡ng 300nits, MÃ n hÃ¬nh ch?ng chÃ³i, ?? p...","57Wh, 135W Slim Tip (3-pin)",28,Intel,Unknown,0,40.0


In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder

# ==============================================================================
# 1. LOAD DATA & XỬ LÝ LỖI FONT CHỮ
# ==============================================================================
filename = 'data_full.csv'
try:
    df = pd.read_csv(filename, header=1, encoding='latin1')
except:
    try:
        df = pd.read_csv(filename, header=1, encoding='utf-16')
    except:
        df = pd.read_csv(filename, header=1, encoding_errors='replace')

print(f"Đã load {len(df)} dòng dữ liệu.")

# ==============================================================================
# 2. FEATURE EXTRACTION: Trích xuất thông tin CPU từ Title/Detail
# ==============================================================================
def extract_cpu_features(row):
    # Ưu tiên lấy từ cpu_detail, nếu thiếu thì lấy từ title
    text_source = str(row['cpu_detail']) if pd.notna(row['cpu_detail']) and str(row['cpu_detail']).strip() != '' else str(row['title'])
    text = text_source.upper()
    
    # Giá trị mặc định
    info = {
        'raw_brand': 'Unknown',
        'raw_family': 'Unknown',
        'cpu_gen': 0,
        'suffix_score': 1.0 # Mặc định là dòng thường (U/M/G)
    }

    # --- A. XÁC ĐỊNH BRAND ---
    if pd.notna(row['cpu_brand']): 
        info['raw_brand'] = row['cpu_brand']
    elif 'INTEL' in text: info['raw_brand'] = 'Intel'
    elif 'AMD' in text: info['raw_brand'] = 'AMD'
    elif 'APPLE' in text or 'MACBOOK' in text: info['raw_brand'] = 'Apple'

    # --- B. XÁC ĐỊNH FAMILY (Dòng chip) ---
    if 'CORE' in text:
        if 'I9' in text: info['raw_family'] = 'Core i9'
        elif 'I7' in text: info['raw_family'] = 'Core i7'
        elif 'I5' in text: info['raw_family'] = 'Core i5'
        elif 'I3' in text: info['raw_family'] = 'Core i3'
        elif 'ULTRA 9' in text: info['raw_family'] = 'Core Ultra 9'
        elif 'ULTRA 7' in text: info['raw_family'] = 'Core Ultra 7'
        elif 'ULTRA 5' in text: info['raw_family'] = 'Core Ultra 5'
    elif 'RYZEN' in text:
        if '9' in text: info['raw_family'] = 'Ryzen 9'
        elif '7' in text: info['raw_family'] = 'Ryzen 7'
        elif '5' in text: info['raw_family'] = 'Ryzen 5'
        elif '3' in text: info['raw_family'] = 'Ryzen 3'
    elif 'M3' in text: info['raw_family'] = 'M3'
    elif 'M2' in text: info['raw_family'] = 'M2'
    elif 'M1' in text: info['raw_family'] = 'M1'
    elif 'CELERON' in text: info['raw_family'] = 'Celeron'
    elif 'PENTIUM' in text: info['raw_family'] = 'Pentium'

    # --- C. XÁC ĐỊNH GEN (Thế hệ) & SUFFIX (Hậu tố) ---
    
    # 1. Regex tìm Gen
    # Tìm chuỗi số đứng sau tên chip (VD: 12400 -> 12, 5600 -> 5)
    # Pattern: Tìm 4-5 chữ số. Nếu Intel 5 số (12400) -> lấy 2 đầu. Nếu 4 số (7735) -> lấy 1 đầu.
    gen_match = re.search(r'(?:I[3579]|RYZEN\s*[3579]|ULTRA\s*[579])[- ]?(\d{4,5})', text)
    if gen_match:
        digits = gen_match.group(1)
        if len(digits) == 5: # VD: 12400, 13700
             info['cpu_gen'] = int(digits[:2])
        elif len(digits) == 4: # VD: 5600, 7735
             # Với Ryzen 4 số: chữ số đầu là Gen (5xxx -> Gen 5)
             # Với Intel Gen 8/9 4 số: (8250 -> Gen 8)
             info['cpu_gen'] = int(digits[0])
    
    # Apple Gen logic (M1/M2/M3)
    if info['raw_brand'] == 'Apple' and info['raw_family'] != 'Unknown':
        info['cpu_gen'] = int(info['raw_family'][-1]) # Lấy số cuối của M1/M2/M3

    # 2. Regex tìm Suffix Score (Hiệu năng)
    # HX/Max > H/Pro > P > U
    if 'HX' in text or 'MAX' in text: info['suffix_score'] = 1.5
    elif 'HS' in text: info['suffix_score'] = 1.35
    elif 'H' in text and 'HZ' not in text: info['suffix_score'] = 1.3
    elif 'PRO' in text: info['suffix_score'] = 1.3
    elif 'P' in text and 'DISPLAY' not in text: info['suffix_score'] = 1.15
    elif 'U' in text: info['suffix_score'] = 0.9

    return pd.Series(info)

# Áp dụng hàm trích xuất
df_extracted = df.apply(extract_cpu_features, axis=1)
df = pd.concat([df, df_extracted], axis=1)

# ==============================================================================
# 3. ENCODING: CHUYỂN ĐỔI SANG SỐ LIỆU
# ==============================================================================

# --- A. CPU FAMILY & PERFORMANCE SCORE (Ordinal Encoding) ---
# Điểm cơ bản cho từng dòng (Base Score)
family_base_score = {
    'Celeron': 15, 'Pentium': 20,
    'Core i3': 30, 'Ryzen 3': 30,
    'Core i5': 50, 'Ryzen 5': 50, 'Core Ultra 5': 55,
    'Core i7': 70, 'Ryzen 7': 70, 'Core Ultra 7': 75,
    'Core i9': 90, 'Ryzen 9': 90, 'Core Ultra 9': 95,
    'M1': 60, 'M2': 75, 'M3': 85,
    'Unknown': 40 # Điểm trung bình mặc định
}

# Tính điểm tổng hợp: (Base Score + Gen Bonus) * Suffix Multiplier
# Gen Bonus: Mỗi đời mới cộng thêm 3 điểm
df['base_score'] = df['raw_family'].map(family_base_score).fillna(40)
df['cpu_performance_score'] = (df['base_score'] + df['cpu_gen'] * 3) * df['suffix_score']
df['cpu_performance_score'] = df['cpu_performance_score'].round(2)

# --- B. CPU BRAND (One-Hot Encoding thủ công để giữ NaN) ---
def encode_brand_custom(val):
    val = str(val).lower()
    if 'intel' in val: return [1, 0, 0]
    elif 'amd' in val: return [0, 1, 0]
    elif 'apple' in val: return [0, 0, 1]
    else: return [np.nan, np.nan, np.nan] # Giữ NaN cho Unknown

brand_cols = ['brand_Intel', 'brand_AMD', 'brand_Apple']
df[brand_cols] = df['raw_brand'].apply(lambda x: pd.Series(encode_brand_custom(x)))

# ==============================================================================
# 4. KẾT QUẢ & LƯU FILE
# ==============================================================================

# Chọn các cột cần thiết cho mô hình
final_columns = [
    'title', 'price_sale',         # Thông tin gốc
    'brand_Intel', 'brand_AMD', 'brand_Apple', # One-Hot Brand (Số 0/1)
    'cpu_performance_score',       # Điểm hiệu năng tổng hợp (Số thực)
    'cpu_gen'                      # Thế hệ chip (Số nguyên)
]

print("\n--- MẪU DỮ LIỆU SAU KHI XỬ LÝ ---")
print(df[final_columns].head(10))

print("\n--- KIỂM TRA THÔNG TIN ---")
print(df[['raw_family', 'cpu_gen', 'suffix_score', 'cpu_performance_score']].head(10))

# Lưu file CSV hoàn chỉnh để train
df.to_csv('data_processed_complete.csv', index=False)
print("\nĐã lưu file: data_processed_complete.csv")

# Score = (Base_Family + Gen * 3) * Suffix

'''
Xử lý cpu_gen (Thế hệ): Code đã thêm logic Regex để bắt được số thế hệ 
(ví dụ từ chuỗi "Core i5 12400" sẽ lấy được gen=12). Điều này cực kỳ quan trọng vì i5 Gen 12
mạnh hơn hẳn i5 Gen 10.

Hệ số suffix_score: Phân biệt được chip dòng U (tiết kiệm điện - hệ số 0.9) và chip dòng H/HX (hiệu năng cao - hệ số 1.3 ~ 1.5).

Công thức điểm cpu_performance_score: Được tính toán khoa học:
    Score = (Base_Family + Gen * 3) * Suffix
Ví dụ: Core i5 (50đ) Gen 12 (+36đ) dòng H (x1.3) = 111.8 điểm.
Con số này phản ánh rất sát giá trị thực tế của CPU để Linear Regression học được.
'''

Đã load 6371 dòng dữ liệu.


KeyError: 'cpu_detail'

In [9]:
import pandas as pd
import numpy as np

# Load file dữ liệu đã xử lý CPU ở bước trước
df = pd.read_csv('data_processed_complete.csv')

# 1. XỬ LÝ CỘT GIÁ (Cleaning Price)
# Loại bỏ dấu chấm (.) và dấu hỏi (?), chuyển thành số
cols_price = ['price_sale', 'price_base']
for col in cols_price:
    df[col] = df[col].astype(str).str.replace(r'[.?]', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce') # Chuyển lỗi thành NaN

# 2. XỬ LÝ CỘT BRAND (One-Hot Encoding)
# Chuẩn hóa chữ hoa/thường (Title Case)
df['brand'] = df['brand'].str.title()

# One-Hot Encoding: Tạo các cột brand_Dell, brand_Asus...
# dtype=int để ra số 0/1 thay vì True/False
df_brand_encoded = pd.get_dummies(df['brand'], prefix='brand', dummy_na=False, dtype=int)

# Ghép nối vào DataFrame chính và bỏ cột 'brand' cũ
df_ready = pd.concat([df, df_brand_encoded], axis=1).drop(columns=['brand'])

# 3. KIỂM TRA & LƯU
print("Các cột Brand mới:", [c for c in df_ready.columns if 'brand_' in c])
print(df_ready[['title', 'price_sale', 'brand_Dell', 'brand_Asus']].head())

df_ready.to_csv('data_final_training.csv', index=False)

Các cột Brand mới: ['brand_Intel', 'brand_AMD', 'brand_Apple', 'brand_Acer', 'brand_Alienware', 'brand_Asus', 'brand_Avita', 'brand_Brand', 'brand_Dell', 'brand_Gaming', 'brand_Gigabyte', 'brand_Hp', 'brand_Huawei', 'brand_Lenovo', 'brand_Lg', 'brand_Masstel', 'brand_Msi', 'brand_Samsung', 'brand_Surface', 'brand_Vaio', 'brand_Xiaomi']
                                               title  price_sale  brand_Dell  \
0          Laptop ASUS TUF Gaming F16 FX607VJ-RL034W  22490000.0           0   
1  Laptop Gaming Acer Nitro V 15 ProPanel ANV15-4...  25990000.0           0   
2     Laptop ASUS Gaming Vivobook 16X K3605VC-RP431W  19490000.0           0   
3                      Laptop HP 14-EP0112TU 8C5L1PA  15490000.0           0   
4                Laptop Lenovo V15 G4 IRU 83A100PMVN  14990000.0           0   

   brand_Asus  
0           1  
1           0  
2           1  
3           0  
4           0  


In [10]:
import pandas as pd
import numpy as np

# Load dữ liệu đã xử lý CPU ở bước trước
df = pd.read_csv('data_processed_complete.csv')

# =========================================================
# 1. XỬ LÝ CỘT GIÁ (Cleaning Price) - QUAN TRỌNG LÀM TRƯỚC
# =========================================================
# Xóa mọi ký tự không phải số (\D) như dấu chấm, dấu hỏi, chữ đ
cols_price = ['price_sale', 'price_base']
for col in cols_price:
    df[col] = df[col].astype(str).str.replace(r'\D', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce') # Chuyển thành số thực

# Loại bỏ các dòng không có giá bán (nếu có) để tránh lỗi tính toán
df_train = df.dropna(subset=['price_sale']).copy()

# =========================================================
# 2. TARGET ENCODING CHO CỘT BRAND (Thay thế One-Hot)
# =========================================================
# Chuẩn hóa tên hãng
df_train['brand'] = df_train['brand'].str.title()

# Bước 1: Tính giá trung bình cho mỗi hãng (Mean Price per Brand)
brand_stats = df_train.groupby('brand')['price_sale'].mean()

# Bước 2: Map giá trị trung bình vào dataframe (Tạo cột brand_score)
df_train['brand_score'] = df_train['brand'].map(brand_stats)

# Bước 3: Xử lý hãng lạ (nếu có trong tương lai) bằng giá trung bình toàn cục
global_mean_price = df_train['price_sale'].mean()
df_train['brand_score'] = df_train['brand_score'].fillna(global_mean_price)

# Bước 4: Chia nhỏ số liệu để mô hình dễ học (Đơn vị: Triệu đồng)
# Ví dụ: 25.000.000 -> 25.0
df_train['brand_score'] = df_train['brand_score'] / 1_000_000

# =========================================================
# 3. LƯU KẾT QUẢ TỐI ƯU
# =========================================================
# Chọn các cột cần thiết, loại bỏ cột brand gốc (chữ) và brand one-hot (nếu có)
final_cols = [
    'title', 'price_sale', 'price_base',
    'brand_score',               # <-- Đặc trưng mới thay cho Brand (1 cột)
    'cpu_performance_score',     # Điểm hiệu năng CPU
    'cpu_gen',                   # Thế hệ CPU
    'brand_Intel', 'brand_AMD', 'brand_Apple', # CPU Brand (3 cột - giữ nguyên vì ít)
    'screen_size'                # Nếu có (cần làm sạch tương tự)
]

# Kiểm tra nếu có screen_size thì giữ lại
if 'screen_size' in df_train.columns:
    # Làm sạch nhanh screen_size (bỏ chữ inches)
    df_train['screen_size'] = df_train['screen_size'].astype(str).str.replace(r'[^0-9.]', '', regex=True)
    df_train['screen_size'] = pd.to_numeric(df_train['screen_size'], errors='coerce')
else:
    final_cols.remove('screen_size')

# Lọc cột tồn tại
available_cols = [c for c in final_cols if c in df_train.columns]
df_final = df_train[available_cols]

print("Bảng giá trị đại diện cho các hãng (Triệu VNĐ):")
print((brand_stats / 1_000_000).head())

print("\nDữ liệu mẫu sau khi encoding:")
print(df_final.head())

# Lưu file
df_final.to_csv('data_optimized_encoding.csv', index=False)

Bảng giá trị đại diện cho các hãng (Triệu VNĐ):
brand
Acer      16.464359
Asus      22.203273
Avita      7.990000
Dell      15.859821
Gaming    16.808750
Name: price_sale, dtype: float64

Dữ liệu mẫu sau khi encoding:
                                               title  price_sale  price_base  \
0          Laptop ASUS TUF Gaming F16 FX607VJ-RL034W  22490000.0  24490000.0   
1  Laptop Gaming Acer Nitro V 15 ProPanel ANV15-4...  25990000.0  26990000.0   
2     Laptop ASUS Gaming Vivobook 16X K3605VC-RP431W  19490000.0  22090000.0   
3                      Laptop HP 14-EP0112TU 8C5L1PA  15490000.0  19590000.0   
4                Laptop Lenovo V15 G4 IRU 83A100PMVN  14990000.0  15990000.0   

   brand_score  cpu_performance_score  cpu_gen  brand_Intel  brand_AMD  \
0    22.203273                  46.00        0          1.0        0.0   
1    16.808750                 122.85        7          0.0        1.0   
2    22.203273                  46.00        0          1.0        0.0   
3    

In [11]:
import pandas as pd
import numpy as np
import re

# ==============================================================================
# 1. LOAD DỮ LIỆU & TỰ ĐỘNG SỬA LỖI HEADER
# ==============================================================================
try:
    df = pd.read_csv('data_full.csv', header=0, encoding='latin1')
except:
    df = pd.read_csv('data_full.csv', header=0, encoding_errors='replace')

# Kiểm tra nếu header bị đẩy xuống dòng 1 (do dòng 0 chứa rác 'Column1')
first_row = df.iloc[0].astype(str).tolist()
if 'price_sale' not in df.columns and any('price_sale' in v for v in first_row):
    print("Phát hiện tiêu đề nằm ở dòng dữ liệu đầu tiên. Đang sửa...")
    df.columns = df.iloc[0] # Lấy dòng 0 làm header
    df = df[1:].reset_index(drop=True) # Xóa dòng 0 thừa

print("Các cột gốc được giữ lại:", df.columns.tolist())

# ==============================================================================
# 2. LÀM SẠCH DỮ LIỆU TRÊN CỘT GỐC
# ==============================================================================

# A. Cột Giá: Bỏ dấu chấm, dấu ?, chuyển thành số
for col in ['price_sale', 'price_base']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(r'\D', '', regex=True)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Xóa các dòng không có giá bán (Target Variable)
if 'price_sale' in df.columns:
    df = df.dropna(subset=['price_sale']).reset_index(drop=True)

# B. Cột Màn hình: Lấy số (15.6), bỏ chữ (inches)
def clean_screen_size(val):
    if pd.isna(val): return np.nan
    val_str = str(val).lower().replace('inches', '').replace('inch', '').strip()
    try:
        match = re.search(r'(\d+\.?\d*)', val_str)
        if match: return float(match.group(1))
    except: pass
    return np.nan

if 'screen_size' in df.columns:
    df['screen_size'] = df['screen_size'].apply(clean_screen_size)
    df['screen_size'] = df['screen_size'].fillna(df['screen_size'].median())

# ==============================================================================
# 3. TẠO CÁC CỘT ENCODING MỚI (FEATURE ENGINEERING)
# ==============================================================================

# Hàm trích xuất thông tin CPU (dùng cho bước tạo cột mới)
def extract_cpu_info(row):
    t1 = str(row.get('cpu_detail', ''))
    t2 = str(row.get('title', ''))
    text = (t1 if pd.notna(t1) and t1.strip() != '' else t2).upper()
    
    info = {'cpu_brand_clean': 'Unknown', 'cpu_gen': 0, 'suffix_score': 1.0, 'raw_family': 'Unknown'}

    # 1. Xác định Brand
    c_brand = str(row.get('cpu_brand', '')).upper()
    if 'INTEL' in c_brand: info['cpu_brand_clean'] = 'Intel'
    elif 'AMD' in c_brand: info['cpu_brand_clean'] = 'AMD'
    elif 'APPLE' in c_brand: info['cpu_brand_clean'] = 'Apple'
    if info['cpu_brand_clean'] == 'Unknown':
        if 'INTEL' in text: info['cpu_brand_clean'] = 'Intel'
        elif 'AMD' in text: info['cpu_brand_clean'] = 'AMD'
        elif 'APPLE' in text: info['cpu_brand_clean'] = 'Apple'

    # 2. Xác định Family
    if 'CORE' in text:
        if 'I9' in text: info['raw_family'] = 'Core i9'
        elif 'I7' in text: info['raw_family'] = 'Core i7'
        elif 'I5' in text: info['raw_family'] = 'Core i5'
        elif 'I3' in text: info['raw_family'] = 'Core i3'
        elif 'ULTRA 9' in text: info['raw_family'] = 'Core Ultra 9'
        elif 'ULTRA 7' in text: info['raw_family'] = 'Core Ultra 7'
        elif 'ULTRA 5' in text: info['raw_family'] = 'Core Ultra 5'
    elif 'RYZEN' in text:
        if '9' in text: info['raw_family'] = 'Ryzen 9'
        elif '7' in text: info['raw_family'] = 'Ryzen 7'
        elif '5' in text: info['raw_family'] = 'Ryzen 5'
        elif '3' in text: info['raw_family'] = 'Ryzen 3'
    elif 'M3' in text: info['raw_family'] = 'M3'
    elif 'M2' in text: info['raw_family'] = 'M2'
    elif 'M1' in text: info['raw_family'] = 'M1'
    
    # 3. Xác định Gen
    match_gen = re.search(r'(?:I[3579]|RYZEN\s*[3579]|ULTRA\s*[579])[- ]?(\d{4,5})', text)
    if match_gen:
        d = match_gen.group(1)
        if len(d) == 5: info['cpu_gen'] = int(d[:2])
        elif len(d) == 4: info['cpu_gen'] = int(d[0])
    
    if info['cpu_brand_clean'] == 'Apple' and info['raw_family'] != 'Unknown':
        try: info['cpu_gen'] = int(info['raw_family'][-1])
        except: pass
        
    # 4. Xác định Suffix
    if 'HX' in text or 'MAX' in text: info['suffix_score'] = 1.5
    elif 'HS' in text: info['suffix_score'] = 1.35
    elif 'H' in text and 'HZ' not in text: info['suffix_score'] = 1.3
    elif 'P' in text and 'DISPLAY' not in text: info['suffix_score'] = 1.15
    elif 'U' in text: info['suffix_score'] = 0.9

    return pd.Series(info)

# Áp dụng trích xuất (tạo DataFrame tạm thời)
df_features = df.apply(extract_cpu_info, axis=1)

# Tính điểm hiệu năng CPU
family_base = {
    'Core i3': 30, 'Ryzen 3': 30, 'Core i5': 50, 'Ryzen 5': 50, 'Core Ultra 5': 55,
    'Core i7': 70, 'Ryzen 7': 70, 'Core Ultra 7': 75,
    'Core i9': 90, 'Ryzen 9': 90, 'Core Ultra 9': 95,
    'M1': 60, 'M2': 75, 'M3': 85, 'Unknown': 40
}
base_score = df_features['raw_family'].map(family_base).fillna(40)
cpu_perf_score = ((base_score + df_features['cpu_gen'] * 3) * df_features['suffix_score']).round(2)

# One-Hot Encoding cho CPU Brand
def encode_cpu(val):
    if val == 'Intel': return [1, 0, 0]
    elif val == 'AMD': return [0, 1, 0]
    elif val == 'Apple': return [0, 0, 1]
    return [0, 0, 0]
cpu_ohe = df_features['cpu_brand_clean'].apply(lambda x: pd.Series(encode_cpu(x), index=['cpu_brand_Intel', 'cpu_brand_AMD', 'cpu_brand_Apple']))

# Target Encoding cho Laptop Brand
brand_series = df['brand'].str.title().fillna('Unknown')
brand_mean = df.groupby(brand_series)['price_sale'].mean()
global_mean = df['price_sale'].mean()
brand_score = brand_series.map(brand_mean).fillna(global_mean) / 1_000_000

# ==============================================================================
# 4. GHÉP CỘT MỚI VÀO SAU CÙNG & LƯU FILE
# ==============================================================================

# Tạo DataFrame chứa các cột mới
new_cols = pd.DataFrame({
    'brand_score': brand_score,              # Điểm số thương hiệu
    'cpu_gen': df_features['cpu_gen'],       # Thế hệ CPU
    'cpu_performance_score': cpu_perf_score  # Điểm sức mạnh CPU
})
# Ghép thêm cột One-Hot CPU
new_cols = pd.concat([new_cols, cpu_ohe], axis=1)

# Nối vào sau DataFrame gốc (giữ nguyên gốc)
df_final = pd.concat([df, new_cols], axis=1)

print("\nĐã xử lý xong. Các cột Encoding nằm ở cuối:")
print(df_final.columns.tolist()[-6:]) # In thử 6 cột cuối cùng
print("\nMẫu dữ liệu:")
print(df_final[['price_sale', 'brand', 'brand_score', 'cpu_performance_score']].head())

df_final.to_csv('data_full_encoded.csv', index=False)

Các cột gốc được giữ lại: ['link', 'title', 'price_sale', 'price_base', 'brand', 'cpu_brand', 'cpu_detail', 'cpu_cores', 'cpu_threads', 'ram_size', 'ram_type', 'storage_size', 'storage_type', 'gpu_info', 'screen_size', 'screen_res', 'screen_tech', 'battery', 'raw_specs_count']

Đã xử lý xong. Các cột Encoding nằm ở cuối:
['brand_score', 'cpu_gen', 'cpu_performance_score', 'cpu_brand_Intel', 'cpu_brand_AMD', 'cpu_brand_Apple']

Mẫu dữ liệu:
   price_sale   brand  brand_score  cpu_performance_score
0  22490000.0    ASUS    19.231018                  40.00
1  25990000.0  Gaming    15.823542                 122.85
2  19490000.0    ASUS    19.231018                  40.00
3  15490000.0      HP    13.070087                  50.00
4  14990000.0  Lenovo    18.090510                  40.00


PermissionError: [Errno 13] Permission denied: 'data_full_encoded.csv'

In [2]:
import pandas as pd
import numpy as np
import re

# ==============================================================================
# 1. LOAD DỮ LIỆU TỪ FILE MỚI
# ==============================================================================
filename = 'data_full_final2.csv'
try:
    df = pd.read_csv(filename, header=0, encoding='latin1')
except:
    df = pd.read_csv(filename, header=0, encoding_errors='replace')

print(f"Đã load file {filename}. Các cột hiện có: {len(df.columns)}")

# ==============================================================================
# 2. LÀM SẠCH DỮ LIỆU (Giữ nguyên các cột khác)
# ==============================================================================

# A. Xử lý cột Giá (Ưu tiên dùng price_sale_clean nếu có, hoặc clean lại từ price_sale)
# Để đồng bộ, tôi sẽ clean trực tiếp trên cột price_sale để chuyển thành số
for col in ['price_sale', 'price_base']:
    if col in df.columns:
        # Bỏ chữ đ, dấu chấm, ký tự lạ
        df[col] = df[col].astype(str).str.replace(r'\D', '', regex=True)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# B. Xử lý Màn hình (Screen Size)
def clean_screen_size(val):
    if pd.isna(val): return np.nan
    val_str = str(val).lower().replace('inches', '').replace('inch', '').strip()
    try:
        match = re.search(r'(\d+\.?\d*)', val_str)
        if match: return float(match.group(1))
    except: pass
    return np.nan

if 'screen_size' in df.columns:
    df['screen_size'] = df['screen_size'].apply(clean_screen_size)
    df['screen_size'] = df['screen_size'].fillna(df['screen_size'].median())

# ==============================================================================
# 3. FEATURE ENGINEERING: CPU
# ==============================================================================
def extract_cpu_features(row):
    t1 = str(row.get('cpu_detail', ''))
    t2 = str(row.get('title', ''))
    text = (t1 if pd.notna(t1) and t1.strip() != '' else t2).upper()
    
    info = {'cpu_brand_clean': 'Unknown', 'cpu_gen': 0, 'suffix_score': 1.0, 'raw_family': 'Unknown'}

    # 1. Brand
    c_brand = str(row.get('cpu_brand', '')).upper()
    if 'INTEL' in c_brand: info['cpu_brand_clean'] = 'Intel'
    elif 'AMD' in c_brand: info['cpu_brand_clean'] = 'AMD'
    elif 'APPLE' in c_brand: info['cpu_brand_clean'] = 'Apple'
    
    if info['cpu_brand_clean'] == 'Unknown':
        if 'INTEL' in text: info['cpu_brand_clean'] = 'Intel'
        elif 'AMD' in text: info['cpu_brand_clean'] = 'AMD'
        elif 'APPLE' in text: info['cpu_brand_clean'] = 'Apple'

    # 2. Family
    if 'CORE' in text:
        if 'I9' in text: info['raw_family'] = 'Core i9'
        elif 'I7' in text: info['raw_family'] = 'Core i7'
        elif 'I5' in text: info['raw_family'] = 'Core i5'
        elif 'I3' in text: info['raw_family'] = 'Core i3'
        elif 'ULTRA 9' in text: info['raw_family'] = 'Core Ultra 9'
        elif 'ULTRA 7' in text: info['raw_family'] = 'Core Ultra 7'
        elif 'ULTRA 5' in text: info['raw_family'] = 'Core Ultra 5'
    elif 'RYZEN' in text:
        if '9' in text: info['raw_family'] = 'Ryzen 9'
        elif '7' in text: info['raw_family'] = 'Ryzen 7'
        elif '5' in text: info['raw_family'] = 'Ryzen 5'
        elif '3' in text: info['raw_family'] = 'Ryzen 3'
    elif 'M3' in text: info['raw_family'] = 'M3'
    elif 'M2' in text: info['raw_family'] = 'M2'
    elif 'M1' in text: info['raw_family'] = 'M1'
    
    # 3. Gen
    match_gen = re.search(r'(?:I[3579]|RYZEN\s*[3579]|ULTRA\s*[579])[- ]?(\d{4,5})', text)
    if match_gen:
        d = match_gen.group(1)
        if len(d) == 5: info['cpu_gen'] = int(d[:2])
        elif len(d) == 4: info['cpu_gen'] = int(d[0])
    
    if info['cpu_brand_clean'] == 'Apple' and info['raw_family'] != 'Unknown':
        try: info['cpu_gen'] = int(info['raw_family'][-1])
        except: pass
        
    # 4. Suffix
    if 'HX' in text or 'MAX' in text: info['suffix_score'] = 1.5
    elif 'HS' in text: info['suffix_score'] = 1.35
    elif 'H' in text and 'HZ' not in text: info['suffix_score'] = 1.3
    elif 'P' in text and 'DISPLAY' not in text: info['suffix_score'] = 1.15
    elif 'U' in text: info['suffix_score'] = 0.9

    return pd.Series(info)

# Apply Feature Extraction
df_cpu = df.apply(extract_cpu_features, axis=1)

# Scoring
family_base = {
    'Core i3': 30, 'Ryzen 3': 30, 'Core i5': 50, 'Ryzen 5': 50, 'Core Ultra 5': 55,
    'Core i7': 70, 'Ryzen 7': 70, 'Core Ultra 7': 75,
    'Core i9': 90, 'Ryzen 9': 90, 'Core Ultra 9': 95,
    'M1': 60, 'M2': 75, 'M3': 85, 'Unknown': 40
}
base_score = df_cpu['raw_family'].map(family_base).fillna(40)
cpu_performance_score = ((base_score + df_cpu['cpu_gen'] * 3) * df_cpu['suffix_score']).round(2)

# ==============================================================================
# 4. ENCODING
# ==============================================================================

# A. One-Hot CPU Brand
def encode_cpu(val):
    if val == 'Intel': return [1, 0, 0]
    elif val == 'AMD': return [0, 1, 0]
    elif val == 'Apple': return [0, 0, 1]
    return [0, 0, 0]

cpu_ohe = df_cpu['cpu_brand_clean'].apply(lambda x: pd.Series(encode_cpu(x), index=['cpu_brand_Intel', 'cpu_brand_AMD', 'cpu_brand_Apple']))

# B. Target Encoding Laptop Brand
# Clean brand name
brand_series = df['brand'].str.title().fillna('Unknown')
# Tính giá trung bình (Dùng price_sale đã clean)
# Xử lý NaN trong price_sale tạm thời để tính toán (nhưng không drop row)
temp_price = df['price_sale'].fillna(df['price_sale'].mean())
brand_mean = df.groupby(brand_series)['price_sale'].mean()
global_mean = df['price_sale'].mean()

brand_score = brand_series.map(brand_mean).fillna(global_mean) / 1_000_000

# ==============================================================================
# 5. GHÉP CỘT & LƯU FILE
# ==============================================================================
# Tạo DataFrame các cột mới
new_features = pd.DataFrame({
    'brand_score': brand_score,
    'cpu_gen': df_cpu['cpu_gen'],
    'cpu_performance_score': cpu_performance_score
})
new_features = pd.concat([new_features, cpu_ohe], axis=1)

# Nối vào DataFrame gốc (df giữ nguyên các cột cũ + cột mới ở cuối)
df_final = pd.concat([df, new_features], axis=1)

print("\nĐã xử lý xong. Các cột mới thêm vào:")
print(df_final.columns.tolist()[-6:]) # 6 cột cuối
print(df_final[['brand', 'brand_score', 'cpu_performance_score']].head())

df_final.to_csv('data_full_final2_encoded.csv', index=False)

Đã load file data_full_final2.csv. Các cột hiện có: 25

Đã xử lý xong. Các cột mới thêm vào:
['brand_score', 'cpu_gen', 'cpu_performance_score', 'cpu_brand_Intel', 'cpu_brand_AMD', 'cpu_brand_Apple']
    brand  brand_score  cpu_performance_score
0    ASUS    19.129989                  40.00
1  Gaming    16.097664                 122.85
2    ASUS    19.129989                  40.00
3      HP    14.255502                  50.00
4  Lenovo    18.146762                  40.00
